In [ ]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp
from mediapipe.python.solutions import holistic
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,Bidirectional,BatchNormalization
from keras.callbacks import TensorBoard
from keras.regularizers import l2
from sklearn.metrics import multilabel_confusion_matrix, accuracy_score


In [ ]:
mp_drawing=mp.solutions.drawing_utils   #holistic model
mp_holistic=mp.solutions.holistic       #drawing utilities
def mediapipe_detection(image,model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  #color conversion BGR to RGB
    image.flags.writeable = False                   #Image is no longer writeable
    results = model.process(image)                  # make prediction
    image.flags.writeable = True                    #image is now writeable
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)  #color conversion again RGB to BGR
    return image,results

def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)  # Draw face connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)  # Draw pose connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) #Draw right hand connections

def draw_style_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                              mp_drawing.DrawingSpec(color=(175,13,13), thickness=1, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=1, circle_radius=1)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(178,16,16), thickness=2, circle_radius=4),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=2)    #color line
                              )
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                              mp_drawing.DrawingSpec(color=(255,255,255), thickness=2, circle_radius=1),   #color dot
                              mp_drawing.DrawingSpec(color=(0,0,0), thickness=2, circle_radius=2)    #color line
                              )
def extract_keypoints(results):
    pose = np.array([[res.x,res.y,res.y,res.visibility] for res in results.pose_landmarks.landmark]).flatten() if results.pose_landmarks else np.zeros(33*4)
    face = np.array([[res.x,res.y,res.y] for res in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3)
    lh = np.array([[res.x,res.y,res.y] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x,res.y,res.y] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([pose,face,lh,rh])

In [ ]:
#path for exported data, numpy arrays
DATA_PATH = os.path.join('D:\Data collection\Frame_Data')
# IMAGES_PATH= os.path.join('Frame_collection')
#action that we are creating and detect
# actions=np.array(['Aausadi','Ambulance','Bathroom','Be Careful','Bleeding','Call','Dhanebad','Doctor','Dont Understand','Eklopan',
#                   'Emergency','Firstaid','Good Morning','Happy','Heart_attack','Hello','Help','Hospital','Name','Need','Nice To Meet You',
#                   'Oxygen','Pain','Please','Police','Relax','Sign','Slowly','Sorry','What','Yes','You'])
# actions=np.array(['Ambulance','Bathroom','Be Careful','Bleeding','Good Morning','Help','Name','Relax','Slowly','What'])
# actions=np.array(['Doctor','Dont Understand','Happy','Hello','Hospital','Nice To Meet You','Oxygen','Pain','Police','Yes','Sorry'])
actions=np.array(['Aausadhi','Call','Dhanebad','Eklopan','Emergency','Heart_attack','Need','Please','Prathamik Upachar','Sign','You'])
# actions=np.array(['Doctor','Dont Understand','Happy','Hello','Hospital','Nice To Meet You','Oxygen','Pain','Police','Yes','Sorry','चिकित्सक'])

#thirty videos worth of data
no_sequences = 50

#videos are going to be 60 frames in length
sequence_length = 60
label_map= {label:num for num, label in enumerate(actions)}


In [ ]:
import os

sequences, labels = [], []

# Iterate through actions
for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    label = label_map[action]

    # Iterate through sequences
    for sequence in range(no_sequences):
        window = []

        # Load all frames of the sequence at once
        sequence_frames = [np.load(os.path.join(action_path, str(sequence), "{}.npy".format(frame_num))) 
                           for frame_num in range(sequence_length)]

        # Append the sequence frames to the window
        window.extend(sequence_frames)

        # Append the window to the sequences list
        sequences.append(window)

        # Append the label to the labels list
        labels.append(label)


In [ ]:
X = np.array(sequences)
y = to_categorical(labels).astype(int)


In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)
from sklearn.model_selection import train_test_split

# Assuming you have X (features) and y (labels)
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.25, random_state=42)

# Now X_train and y_train are the training set, and X_val and y_val are the validation set


In [ ]:
print("Length of data in X_train:", len(X_train))
print("Length of data in X_val:", len(X_val))
print("Length of data in Y_train:", len(y_train))
print("Length of data in Y_val:", len(y_val))

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense, Dropout,Bidirectional,BatchNormalization
from keras.callbacks import TensorBoard
from keras.regularizers import l2

In [ ]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [ ]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 1662)))

model.add(Dense(32, activation='relu'))  # Additional Dense layer
model.add(Dense(actions.shape[0], activation='softmax'))
model.summary()

#try second model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60,1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

#try third model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(256, return_sequences=True, activation='relu'))  # Additional LSTM layer
model.add(LSTM(128, return_sequences=False, activation='relu'))
# model.add(LSTM(64, return_sequences=True, activation='relu', kernel_regularizer=l2(0.01)))
model.add(Dense(128, activation='relu'))  # Additional Dense layer
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

#try fourth model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(60, 1662)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout to prevent overfitting
model.add(Dense(32, activation='relu'))
model.add(Dropout(0.5))  # Adding dropout to prevent overfitting
model.add(Dense(actions.shape[0], activation='softmax'))

#try fifth model
model = Sequential()
model.add(Bidirectional(LSTM(128, return_sequences=True, activation='relu'), input_shape=(60, 1662)))
model.add(Bidirectional(LSTM(264, return_sequences=True, activation='relu')))
model.add(LSTM(128, return_sequences=False, activation='relu'))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))  # Adjust dropout rate based on experimentation
model.add(BatchNormalization())
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(actions.shape[0], activation='softmax'))



In [ ]:
# model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])


from keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['categorical_accuracy'])



In [ ]:
from keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)

In [ ]:
# model.fit(X_train, y_train, epochs=300, callbacks=[tb_callback])
history=model.fit(X_train, y_train, epochs=250, batch_size=128, validation_data=(X_val, y_val), callbacks=[tb_callback])
model.save('lstm3.h5')



In [ ]:
from keras.models import load_model

#Provide the path to your saved model
model_path = r'D:\Hand Gesture Recognition\lstm3.h5'
#Load the model
loaded_model = load_model(model_path)


In [ ]:
fig = plt.figure()
plt.plot(history.history['loss'], color='teal', label='loss')
plt.plot(history.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper right")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(history.history['categorical_accuracy'], color='teal', label='accuracy')
plt.plot(history.history['val_categorical_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
yhat = loaded_model.predict(X_train)
ytrue = np.argmax(y_train, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()
multilabel_confusion_matrix(ytrue,yhat)
from sklearn.metrics import confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt


In [ ]:
# #Assuming you have already trained your model and obtained predictions
# y_true are the true labels, and y_pred are the predicted labels
# Replace y_true and y_pred with your actual data
# Compute the confusion matrix
conf_matrix = confusion_matrix(ytrue, yhat)

#Plot the confusion matrix using seaborn heatmap
plt.figure(figsize=(10, 8))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues", cbar=False, 
            xticklabels=actions, yticklabels=actions)
plt.xlabel('Predicted labels')
plt.ylabel('True labels')
plt.title('Confusion Matrix')
plt.show()
accuracy_score(ytrue, yhat)



In [ ]:
from sklearn.metrics import classification_report
#Compute classification report
class_report = classification_report(ytrue, yhat)
print("Classification Report:")
print(class_report)

In [ ]:
sequence = []
sentence = []
predictions = []
threshold = 0.9

cap = cv2.VideoCapture(0)
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():
        # Read feed
        ret, frame = cap.read()
        # make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)

        # Draw landmarks
        # draw_landmarks(image, results)  # default landmarks in face hand

        draw_style_landmarks(image, results)  # customized landmarks in face and hand

        # prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        # sequence.insert(0, keypoints)
        sequence = sequence[-60:]
        
        if len(sequence) == 60:
            res = loaded_model.predict(np.expand_dims(sequence, axis=0))[0]
            # print(actions[np.argmax(res)])
            # print("Confidence values:", res)
            print("Predicted Action:", actions[np.argmax(res)])
            predictions.append(np.argmax(res))

            # visualization logic
            if np.unique(predictions[-20:])[0]==np.argmax(res):
                if res[np.argmax(res)] > threshold:
                    if len(sentence) > 0:
                        if actions[np.argmax(res)] != sentence[-1]:
                            sentence.append(actions[np.argmax(res)])
                        else:
                            sentence.append(actions[np.argmax(res)])

            if len(sentence) > 5:
                sentence = sentence[-5:]

        # viz probabilities
            # image = prob_viz(res,actions,image,colors)

        # image = cv2.rectangle(image, (0, 0), (640, 40), (245, 117, 16), -1)
        image = cv2.putText(image, ' '.join(sentence), (3, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2,
                            cv2.LINE_AA)
        # show to screen
        image = cv2.flip(image, 1)
        cv2.imshow('OPENCV FEED', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

In [ ]:
cap.release()
cv2.destroyAllWindows()

In [ ]:
res[np.argmax(res)] >threshold